# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings; warnings.filterwarnings('ignore')
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(width=20, with_test=True, label="ppl train", round=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## Seq2Seqの発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できるのではないか。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプション生成などに応用できそう。隠れ状態を通じてRNNからCNNまで逆伝播が繋がるので、画像と文章のペアさえ用意すれば学習できそう。というかできる[1]。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。

本章ではこの翻訳モデルを作成する。入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。こういった、シーケンスをシーケンスに変換するモデルを *seq2seq (Sequence to Sequence)* と呼ぶ。

seq2seqは以下の二つのRNNから構成される。
- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットを使用する。
- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    'huggingface:iwslt2017/iwslt2017-en-ja',
    data_dir='data',
    split='train'
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample['translation']['ja'].decode()
    en = sample['translation']['en'].decode()
    data_ja.append(ja)
    data_en.append(en)

print('num of data:', len(data_ja), '\n')
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

僕の旅は 14年前に始まりました
My journey started 14 years ago.

際立って成功したキャンペーンです その力に注目してみましょう
Remarkably successful campaign, but notice the power of it.

「ありがとう ご職業は？」
I said, "Great. What do you do?"

アメリカの誇りで包まれた 我々を定義する場所としての地位を 私達は長い間 この国の他所の風景に 譲ってきました グランドキャニオン ヨセミテ イエローストーン
cloaking them with this American pride, places that we now consider to define us: Grand Canyon, Yosemite, Yellowstone.

何をすべきかよく分からず とりあえず 周りの人々に 話しかけたんです 彼は人々にユニークなことを聞きました 「自由になった感想は？」
He doesn't quite know what to do, so he starts going around the crowd and starts talking to people and he says to people in this rather unique way, "How does it feel to be free?"



書き出し

In [5]:
textfile_ja = 'data/iwslt2017_ja.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = 'data/iwslt2017_en.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

データ数を減らしたものも作っておく。

In [6]:
n_data = 50000
data_ja = data_ja[:n_data]
data_en = data_en[:n_data]

textfile_ja = f'data/iwslt2017_ja_{n_data}.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = f'data/iwslt2017_en_{n_data}.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

トークナイザの学習。

In [7]:
pad_id = 3
vocab_size = 8000

tokenizer_prefix_ja = f'models/tokenizer_iwslt2017_ja_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

tokenizer_prefix_en = f'models/tokenizer_iwslt2017_en_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/iwslt2017_ja_50000.txt
  input_format: 
  model_prefix: models/tokenizer_iwslt2017_ja_50000
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privac

### 前処理

データ数を減らした方を使う。

トークナイザ読み込み

In [8]:
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [9]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [10]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。  
また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderの作成。

In [11]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

(torch.Size([32, 69]), torch.Size([32, 87]), torch.Size([32, 87]))


---

## Seq2Seqを用いた翻訳モデル

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。LSTMと線形層で作る。

ここで1つ工夫を加える。普通に作っても動きはするが、学習が上手くいかない可能性がある。

学習時にencoderへ入力されるデータはpaddingされたデータとなる。ここで、paddingされた範囲が多い=padトークンが多く含まれているデータは、padの数が多くなるにつれて、隠れ状態がある一定の値に収束してしまう。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうのだ。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、翻訳を学習できない。

そこで、encoderが出力する隠れ状態として、padトークンを除いた最後のトークン=EOSを入力した時点のものを使用する。  

以上を踏まえ、encoderを以下のように実装する。

In [12]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(
            embed_size,
            hidden_size,
            batch_first=True,
            num_layers=3,
            dropout=0.2
        )
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        x = self.dropout(x)
        _, hc = self.lstm(x) # (batch_size, seq_len, hidden_size)
        # h = hs[eos_positions].unsqueeze(0) # (1, batch_size, hidden_size)
        return hc

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [13]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(
            embed_size,
            hidden_size,
            batch_first=True,
            num_layers=3,
            dropout=0.2
        )
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        x = self.dropout(x)
        hs, hc = self.lstm(x, hc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (3, batch_size, hidden_size)
            # c: (3, batch_size, hidden_size)
        # hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, hc

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        hc = self.encoder(x_enc)
        # hc = (h, torch.zeros_like(h))
        y, _ = self.decoder(x_dec, hc)
        return y

In [20]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 24,903,488



---

## 実践

### 学習

In [24]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    ppls = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        ppl = torch.exp(loss).item()
        ppls.append(ppl)
    ppl = sum(ppls) / len(ppls)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            ppl = torch.exp(loss).item()
            prog.update(ppl)

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [26]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [27]:
train(model, optimizer, n_epochs=20, prog_unit=1)

 1/20: #################### 100% [00:07:57.28] ppl train: 500.66, test: 285.36 
 2/20: #################### 100% [00:08:04.10] ppl train: 231.94, test: 188.24 
 3/20: #################### 100% [00:07:59.97] ppl train: 175.01, test: 156.47 
 4/20: #################### 100% [00:08:01.69] ppl train: 149.81, test: 139.30 
 5/20: #################### 100% [00:08:04.23] ppl train: 134.13, test: 127.81 
 6/20: #################### 100% [00:08:00.38] ppl train: 122.81, test: 119.26 
 7/20: #################### 100% [00:07:59.79] ppl train: 113.74, test: 112.51 
 8/20: #################### 100% [00:08:00.70] ppl train: 106.42, test: 107.78 
 9/20: #################### 100% [00:08:00.84] ppl train: 100.10, test: 102.64 
10/20: #################### 100% [00:07:59.82] ppl train: 94.69, test: 98.87  
11/20: #################### 100% [00:08:01.15] ppl train: 89.88, test: 96.11 
12/20: #################### 100% [00:08:02.61] ppl train: 85.57, test: 93.35 
13/20: #################### 100% [00:08:00.70

In [36]:
model_path = "models/lm_seq2seq.pth"
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [41]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    hc = model.encoder(in_ids)
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).reshape(1, 1)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [42]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 例えば 人々は見知らぬ人による誘拐を恐れますが データによると顔見知りによる誘拐の方が断然多いのです
output: And if you're going to do a little bit of a few years, you're going to see that the world's spussion, and it's not a very good thing to do, but it's a very important thing to do.
answer: One example would be, people fear kidnapping by strangers when the data supports kidnapping by relatives is much more common.

input: ある私のお気に入りの研究では 夫が家事に積極的なほど 妻は夫に魅力を感じることが判明しました
output: And I was a little bit of a few years ago, and I was a very good idea, and I was a little bit of a few years ago, and I was a little bit of a few years.
answer: One of my favorite studies found that the more willing a husband is to do house work, the more attractive his wife will find him.

input: スローガンを一つ掲げます 「子供の仕事はおもちゃを壊すこと」です
output: And I'm going to show you a little bit of a couple of years ago, and I'm going to say, "I'm going to show you a little bit of a little bit of a day.
answer: And we have a slogan that the best thing a child can do with 

訓練データに含まれていないものも試してみる。

In [43]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: このタペストリーの展示場に設置しました
output: This is a very interesting thing, and this is a very good thing.
answer: And we were put in this tapestry room.

input: 全世界的なレベルでいうと、私たちは自分自身を超えるものを持たなくてはなりません。
output: And we're going to be able to do that, and we're going to be able to do that.
answer: On the global level, we have to have more than our own thing.

input: 制約の中で 創造性を 発揮する術を学ぶことが 私達が自分を変え 全体として世界を変える ― 最も確かな道なのです
output: And if you look at the same time, you can see that the world's population is not a very good thing, and it's a very important thing, and it's a very important thing.
answer: Learning to be creative within the confines of our limitations is the best hope we have to transform ourselves and, collectively, transform our world.

input: しかし、必要な結果が得られませんでした。
output: But we're going to do this.
answer: But they couldn't get the results they wanted.

input: 引っ張って紙タオルを 切り取るタイプもあります
output: And I'm going to show you a lot of people who are in the middle of the world.
answer: Ther

In [44]:
# original
sentences = [
    'ありがとう。',
    '猫はかわいいね。',
    '上手く文章が書けるようになりました。'
]

for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: ありがとう。
output: herently.

input: 猫はかわいいね。
output: s a lot of people.

input: 上手く文章が書けるようになりました。
output: s the world's own.



びみょう。